In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import YXJ
import torch.nn.functional as F

class TimeSeriesDataset2(Dataset):
    def __init__(self, csv_file, window_size, step_size):
        self.data_frame = pd.read_csv(csv_file) # 读取CSV文件
        self.window_size = window_size
        self.step_size = step_size
        self.samples = self._create_samples()

    def _create_samples(self):
        samples = []
        for start_pos in range(0, len(self.data_frame) - self.window_size + 1, self.step_size):
            end_pos = start_pos + self.window_size
            sample = self.data_frame.iloc[start_pos:end_pos].values
            samples.append(sample)
        return np.array(samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        X = sample[:, :-1]  # 所有行，除了最后一列
        y = np.mean(sample[:, -1])  # 最后一列的平均值作为标签

        # 转换为Tensor
        X = torch.tensor(X, dtype=torch.float32)
        X = X.transpose(0, 1)
        y = torch.tensor(y, dtype=torch.float32)
        return X, y

# 使用示例
SAMPLES_PER_GESTURE = 3000
step_size = 100
dataset = TimeSeriesDataset2('normalized_data.csv', SAMPLES_PER_GESTURE,step_size)
GESTURES = ["smoke", "nosmoke"]  # 手势列表
NUM_GESTURES= len(GESTURES);

# 分割数据集
from torch.utils.data import random_split
dataset_size = len(dataset)
train_size = int(dataset_size * 0.7)  # 70% 数据用于训练
valid_size = int(dataset_size * 0.2)  # 20% 数据用于验证
test_size = dataset_size - train_size - valid_size  # 剩余10% 用于测试
batch_size =512;
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

YXJ.check_dataloader(test_loader)
input_size=SAMPLES_PER_GESTURE*9

torch.Size([68, 9, 3000]) tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.3300, 0.3300, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.3230, 0.4507, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0370,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])


In [2]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size=2, dropout=0.2):
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = input_size if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        x = self.network(x)
        x = x[:, :, -1]  # 只取序列的最后一个输出
        return torch.sigmoid(self.linear(x))


In [3]:
def validate(model, device, criterion, test_loader):
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += criterion(output, target).item()
    validation_loss /= len(test_loader)  # 计算平均验证损失
    return validation_loss
    
def train(model, device, train_loader, criterion, optimizer, epochs):
    model.train()
    train_losses = []  # 用于存储每个epoch的损失值
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()           # 清零梯度
            output = model(data)            # 前向传播
            loss = criterion(output, target) # 计算损失
            loss.backward()                 # 反向传播
            optimizer.step()                # 更新参数
        train_losses.append(loss.item())
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return train_losses
    
def train_vali(model, device, train_loader, test_loader, criterion, optimizer, epochs):
    model.train()
    train_losses = []
    validation_losses = []
    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()  # 清零梯度
            output = model(data)  # 前向传播
            loss = criterion(output, target)  # 计算损失
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
        train_losses.append(loss.item())
        
        # 在每个epoch后计算验证集上的损失
        validation_loss = validate(model, device, criterion, test_loader)
        validation_losses.append(validation_loss)
        
        print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {validation_loss}')

    return train_losses, validation_losses
    
def test(model, device, test_loader):
    model.eval()  # 将模型设置为评估模式
    test_loss = 0
    with torch.no_grad():  # 在评估阶段不计算梯度
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # 计算MSE损失，注意这里不需要像之前那样手动计算差异
            # 使用output和target直接计算MSE
            loss = torch.nn.functional.mse_loss(output.squeeze(), target)
            # 累加每个批次的损失
            test_loss += loss.item() * data.size(0)  # 乘以data.size(0)以得到此批次的总损失

    # 计算整个测试集上的平均损失
    test_loss /= len(test_loader.dataset)
    
    print(f'Average MSE Loss on the test set: {test_loss}')

In [4]:
# 使用GPU
use_cuda = torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')

# 模型参数
input_size = 9  # IMU六维+GPS三维
output_size = 1  # 输出是否跑步的概率
num_channels = [25, 50, 100]  # TCN的各层通道数

model = TCN(input_size, output_size, num_channels).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# 假设 train_loader 和 test_loader 已经定义
train_losses, validation_losses=train_vali(model, device, train_loader, valid_loader, criterion, optimizer, epochs=100)
YXJ.draw_loss(train_losses,validation_losses)
test(model, device, valid_loader)
test(model, device, test_loader)

/Users/qiushi11/anaconda3/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


KeyboardInterrupt: 